<a href="https://colab.research.google.com/github/HsuLiSheng/stock-related/blob/main/3_%E6%AF%8F%E6%97%A5%E8%82%A1%E5%83%B9%E5%8D%B3%E6%99%82%E5%A0%B1%E5%83%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install lineTool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for lineTool: filename=lineTool-1.0.3-py3-none-any.whl size=1463 sha256=7ab3a552791bf14b3b497d6bbd20ca3bdfc85f46437272e7cbbe965239274875
  Stored in directory: /root/.cache/pip/wheels/7a/d0/c1/f626ef4b03a3ee37bac2d52f3377138697c2fab58c88af8d10
Successfully built lineTool


In [ ]:
from urllib.request import urlopen
import datetime
import json
import pandas as pd
import lineTool

token = "ZCmEPQ0nXHIB25Pz8IPWTuWPUNkZxL1NOa2IWasGHz6"

In [ ]:
def real_time_inf(targets,end_time):
  
  ###把要觀察的股票代碼轉換形式後代入html中(tse:上市,otc:上櫃)
  stock_list = '|'.join('{}.tw'.format(target) for target in targets)
  ###根據每支股票的當前價格去設定一個突破上下限(+1.5~-1.0)，如果突破會line通知我
  upper_price = [41.6,76.1]
  lower_price = [40.7,75.0]

  html3 = urlopen('https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch='+ stock_list)
  content3 = html3.read().decode('utf-8')
  jcontent3 = json.loads(content3) ###利用json把數據整理更好看
  data3 = jcontent3['msgArray']

  df_data = [[] for i in range(len(data3))]
  ###只選出我們要的欄位(原本共有36個)
  index = ['c','n','z','a','tv','v','o','h','l','y'] 
  for x in range(len(data3)):
    for y in range(len(index)):
      if index[y] == 'a':
         five_price = data3[x][index[y]].split("_")
         for i in range(5):
           df_data[x].append(five_price[i])
      else:   
         df_data[x].append(data3[x][index[y]])


  df_data = pd.DataFrame(df_data)
  df_data.columns = ['股票代號','公司簡稱','當盤成交價','最佳五檔賣出:1','2','3','4','5','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
  
  for i in range(len(data3)):
    if df_data['當盤成交價'].iloc[i] != '-' and float(df_data['當盤成交價'].iloc[i]) > upper_price[i]:
      lineTool.lineNotify(token, df_data.iloc[i][1]+' 當前成交價:'+df_data.iloc[i][2])
    elif df_data['當盤成交價'].iloc[i] != '-' and float(df_data['當盤成交價'].iloc[i]) < lower_price[i]:
      lineTool.lineNotify(token, df_data.iloc[i][1]+' 當前成交價:'+df_data.iloc[i][2])

  now = datetime.datetime.now() + datetime.timedelta(hours = 8)
  print('時間:'+ str(now.hour)+ ':'+ str(now.minute)+ ':'+ str(now.second))
  
  ###每10分鐘把現在每支股票的成交價格印出來
  if now.second >= 30 and now.minute % 5 == 0:
      display(df_data.iloc[:,[1,2,3,4,5,6,7]])
  
  ###設定爬蟲的判斷條件(股票的開始與結束時間)，可以分段設定end時間讓我們可以去更改price的價格
  start = datetime.datetime.strptime(str(now.date())+'9:00','%Y-%m-%d%H:%M')
  end = datetime.datetime.strptime(str(now.date())+end_time,'%Y-%m-%d%H:%M')
  if now >= start and now <= end:
      s.enter(30,0,real_time_inf, argument=(targets,end_time))

In [ ]:
import sched
import time


s = sched.scheduler(time.time, time.sleep)
list1 = ['tse_6443','tse_3022']
end_time = '10:51'
s.enter(30, 0, real_time_inf, argument=(list1,end_time))
s.run()
x = input('延長查詢時間(0 or 1):')
while x == '1':
  end = datetime.datetime.strptime(str(datetime.datetime.now().date())+end_time,'%Y-%m-%d%H:%M')
  overtime = int(input('延長幾分鐘:'))
  end += datetime.timedelta(minutes = overtime)
  end_time = datetime.datetime.strftime(end,'%H:%M')
  print('時間延長至:'+ end_time)
  s.enter(30, 0, real_time_inf, argument=(list1,end_time))
  s.run()
  x = input('延長查詢時間(0 or 1):')

print('.............monitoring is over..............')

時間:10:19:2
時間:10:19:33
時間:10:20:4
時間:10:20:35


,公司簡稱,當盤成交價,最佳五檔賣出:1,2,3,4,5
0,元晶,-,41.1000,41.1500,41.2000,41.2500,41.3000
1,威強電,-,75.8000,75.9000,76.0000,76.1000,76.2000


時間:10:21:6
時間:10:21:36
時間:10:22:7
時間:10:22:38
時間:10:23:8
時間:10:23:39
時間:10:24:10
時間:10:24:41
時間:10:25:11
時間:10:25:42


,公司簡稱,當盤成交價,最佳五檔賣出:1,2,3,4,5
0,元晶,-,41.1000,41.1500,41.2000,41.2500,41.3000
1,威強電,-,76.1000,76.2000,76.3000,76.4000,76.5000


時間:10:26:13
時間:10:26:44
時間:10:27:14
時間:10:27:45
時間:10:28:16
時間:10:28:46
時間:10:29:17
時間:10:29:48
時間:10:30:19
時間:10:30:49


...

時間:10:56:5
時間:10:56:36
時間:10:57:7
時間:10:57:37
時間:10:58:8
時間:10:58:39
時間:10:59:9
時間:10:59:40
時間:11:0:12
時間:11:0:43


,公司簡稱,當盤成交價,最佳五檔賣出:1,2,3,4,5
0,元晶,-,41.1000,41.1500,41.2000,41.2500,41.3000
1,威強電,78.0000,78.0000,78.1000,78.2000,78.3000,78.4000


時間:11:1:13
延長查詢時間(0 or 1):1
延長幾分鐘:5
時間延長至:11:06
時間:11:2:49
時間:11:3:20
時間:11:3:51
時間:11:4:22
時間:11:4:53
時間:11:5:23
時間:11:5:55


,公司簡稱,當盤成交價,最佳五檔賣出:1,2,3,4,5
0,元晶,-,40.9500,41.0000,41.0500,41.1000,41.1500
1,威強電,-,77.7000,77.8000,77.9000,78.0000,78.1000


時間:11:6:26
延長查詢時間(0 or 1):0
.............monitoring is over..............


In [ ]:
now = datetime.datetime.now() + datetime.timedelta(hours = 8)
print(now)
now.minute % 10

2022-09-15 16:10:27.181258


0

In [ ]:
targets = ['otc_6589']
stock_list = '|'.join('{}.tw'.format(target) for target in targets)
###根據每支股票的當前價格去設定一個突破上下限(+1.5~-1.0)，如果突破會line通知我

html3 = urlopen('https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch='+ stock_list)
content3 = html3.read().decode('utf-8')
jcontent3 = json.loads(content3) ###利用json把數據整理更好看
data3 = jcontent3['msgArray']

df_data = [[] for i in range(len(data3))]
###只選出我們要的欄位(原本共有36個)
index = ['c','n','z','a','tv','v','o','h','l','y'] 
for x in range(len(data3)):
  for y in range(len(index)):
    if index[y] == 'a':
       five_price = data3[x][index[y]].split("_")
       for i in range(5):
         df_data[x].append(five_price[i])
       #print(data3[x][index[y]].split("_")[0])
    else:   
      df_data[x].append(data3[x][index[y]])

if len(data3) ==1:
    df_data = pd.DataFrame(df_data)
else:   
    df_data = pd.DataFrame(df_data)    

df_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,6589,台康生技,117.5000,118.0000,118.5000,119.0000,119.5000,120.0000,597,20320,116.5000,125.0000,116.0000,116.0000
